In [8]:
""" connection to data base and extracting data """
 
import pandas as pd
'''
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=BIA-ANALYSIS-DB;'
                      'Database=BIA_Marketing;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query("select top 100 * FROM BIA_Marketing.sas.cust_prod_matrix", conn)
'''
df = pd.read_csv("data_cust_prod_matrix.csv") #.head(10)
df.head()

prod_df = df.iloc[:,1:9] # slecting the product columns excusing user id
prod_mat = prod_df.as_matrix(columns=None) # converting the dataframe into matrix 

C:\Users\u004637\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
# calculating similarity metric using both item and also using customer/user
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(prod_mat, metric='cosine')
item_similarity = pairwise_distances(prod_mat.T, metric='cosine')

print("user_similarity : " + str(user_similarity.shape))
print("item_similarity : " + str(item_similarity.shape))

In [ ]:

""" Function for recommenation matix generation """

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as s_ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred


""" running function against the data matrix """

user_prediction = predict(prod_mat, user_similarity, type='user')
item_prediction = predict(prod_mat, item_similarity, type='item')

print("user_prediction : " + str(user_prediction.shape))
print("item_prediction : " + str(item_prediction.shape))


In [ ]:
'''
user based method accuracy. 
1-MAE is used as accuracy as we can compare and 
also the out put of recommendation ening and input is binary.
This is appropriate metric.

min max transformation. 0 to 1 range

'''

scaled_user_pred = (user_prediction - user_prediction.min()) / (user_prediction.max() - user_prediction.min())
mean = scaled_user_pred.mean()
bi_user_pred = (scaled_user_pred > mean).astype(int)


'''
mean_real = prod_mat.mean()
bi_real = (prod_mat > mean_real).astype(int) # any value greater than mean is flagged as 1 else 0
'''
 
from sklearn.metrics import mean_absolute_error

Accuracy_user_sim = 1- mean_absolute_error(prod_mat, bi_user_pred)


'''
item based method accuracy

'''

scaled_item_pred = (item_prediction - item_prediction.min()) / (item_prediction.max() - item_prediction.min())
mean = scaled_item_pred.mean()
bi_item_pred = (scaled_item_pred > mean).astype(int)

'''
mean_real = prod_mat.mean()
bi_real = (prod_mat > mean_real).astype(int)
'''



In [ ]:
from sklearn.metrics import mean_absolute_error

Accuracy_item_sim = 1- mean_absolute_error(prod_mat, bi_item_pred)

print(Accuracy_item_sim)

print(Accuracy_user_sim)